<a href="https://colab.research.google.com/github/Yuejun-Han/Capstone-Project/blob/main/1_Enformer_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook demonstrates how to prepare the input data for the Enformer model to predict gene expression on a GEUVADIS/1000 genomes dataset.

Enformer's input is a single strand genome sequence. Yet, we are interested in predicting on population level data which includes individual-specific variation. To get around this limitation, we will treat each individual as the sum of their haplotypes. Using the phased variant data around each gene (stored in the variant bed files) to modify the reference sequence, we can create two distinct haplotype sequences for each individual. The sum of both of Enformer's haplotype predictions serves as an individual-specific, additive estimate which we can correlate with true predictions. Together, the files we downloaded give us all the information we need to build these haplotype sequences. 

Although enformer predicts a wide array of functional output, we will focus here on gene expression in lymphoblastoid cells allowing for correlation against ground truth Geuvadis gene expression data.

### Major Steps

- Set up the environment
- Download some the necessary files to begin with
- Randomly select 100 target genes
- Randomly select 120 individuals

## Setup

### Setting up our environments

Import packages that we need later. 

In [ ]:
pip install pyfaidx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 6.6 MB/s 
  Created wheel for pyfaidx: filename=pyfaidx-0.7.0-py3-none-any.whl size=27697 sha256=a5e335178a98b83b989952030add838e6c44848450497b6b17a6aae9c30f1019
  Stored in directory: /root/.cache/pip/wheels/df/6b/ce/46374a70af569061fa10a6c16525b0d8efe2d9a4069f8a144a
Successfully built pyfaidx


In [ ]:
import joblib
import gzip # for manipulating compressed files
import pyfaidx # to index our reference genome file
import pandas as pd # for manipulating dataframes
import numpy as np # for numerical computations
import pickle # for saving large objects
import os, sys # functions for interacting with the operating system
import random  
import glob

%config InlineBackend.figure_format = 'retina'

connect to Google Drive to store results and data pernemently in cloud.   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create a folder called "Capstone_Project_2022" 

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Capstone_Project_2022/results/"):
  !mkdir -p "/content/drive/MyDrive/Capstone_Project_2022/results/"

In [ ]:
local_path = '/content/drive/MyDrive/Capstone_Project_2022/'

In [ ]:
transform_path = 'gs://dm-enformer/models/enformer.finetuned.SAD.robustscaler-PCA500-robustscaler.transform.pkl'
model_path = 'https://tfhub.dev/deepmind/enformer/1'
fasta_file = local_path+'genome.fa'

We may inspect the tracks used to train the model. The CAGE prediction corresponding to B lymphoblastoid cell line is index 5110. We use B lymphoblastoid cell line predictions here because that is the cell line used to generate GEUVADIS gene expression data. You can copy the https link, paste in another tab in your browser and look through the large txt file for other tracks. 

For more information about how Enformer works, check out their [article](https://www.deepmind.com/blog/predicting-gene-expression-with-ai).

In [ ]:
# Download targets from Basenji2 dataset 
# Cite: Kelley et al Cross-species regulatory sequence activity prediction. PLoS Comput. Biol. 16, e1008050 (2020).
targets_txt = 'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_human.txt'
df_targets = pd.read_csv(targets_txt, sep='\t')
df_targets[df_targets.index==5110]

,index,genome,identifier,file,clip,scale,sum_stat,description
5110,5110,0,CNhs12333,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:B lymphoblastoid cell line: GM12878 ENCOD...


# Download Files


We need to download some files to continue our preparation process. Give it a moment. 
We will download the following files:
- The reference genome fasta file (we will also index this file in the process)
- A text file for the transcription start sites for each chromosome 
- Per chromosome files that has annotation for the genes
- A compressed file that contains the variant bed files for the genes and their locations. 

Credit to Genome Reference Consortium: https://www.ncbi.nlm.nih.gov/grc

Schneider et al 2017 http://dx.doi.org/10.1101/gr.213611.116: Evaluation of GRCh38 and de novo haploid genome assemblies demonstrates the enduring quality of the reference assembly

Make a data directory, and download the necessary bed files and chromosome annotation files

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Capstone_Project_2022/data'):
  !mkdir -p '/content/drive/MyDrive/Capstone_Project_2022/data'

The next line of code will download the reference genome fasta file and index this file. 

In [ ]:
# reference genome and indexed
if not os.path.exists('/content/drive/MyDrive/Capstone_Project_2022/genome.fa'):
  !wget -O - https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz | gunzip -c > {fasta_file}
  pyfaidx.Faidx(fasta_file)

#### Download Transcription Starting Site file

In the next block of code, we download the TSS for each chromosome and the genes in that chromosome, as wells as the per chromosome gene annotations. We need this information to estimate predictions. 


In [ ]:
if not os.path.exists("/content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms"):
  !mkdir -p /content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms #creates a folder to hold our files

!curl -L https://uchicago.box.com/shared/static/perc3uabzzd267cbp8zc0inwgrmur7pu.gz --output /content/drive/MyDrive/Capstone_Project_2022/data/chr_tss.tar.xz && cd /content/drive/MyDrive/Capstone_Project_2022/data/ && tar -zxf /content/drive/MyDrive/Capstone_Project_2022/data/chr_tss.tar.xz

!curl -L https://uchicago.box.com/shared/static/e2kiwrjlgqqio0pc37a2iz7l5bqbv57u.gz --output /content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_chroms.tar.gz && cd /content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/ && tar -zxf /content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_chroms.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     7    0     7    0     0      7      0 --:--:-- --:--:-- --:--:--    10
100 1783k  100 1783k    0     0  1154k      0  0:00:01  0:00:01 --:--:-- 76.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     6    0     6    0     0      9      0 --:--:-- --:--:-- --:--:--     9
100  728k  100  728k    0     0   480k      0  0:00:01  0:00:01 --:--:-- 64.8M


# Randomly Select target genes

## Randomly select 100 genes

Randomly select 100 genes from the all_genes.tsv (included in the zipped file chr_tss.gz). This is a tsv file including all the gene names that we have corresponding information.

In [ ]:
with open("/content/drive/MyDrive/Capstone_Project_2022/data/all_genes.tsv", "r") as ag:
  all_genes = [line.strip() for line in ag]
all_genes[0:5]

['PEX10', 'TMEM69', 'ATAD3A', 'BPNT1', 'SNIP1']

now we ranodmly select our 100 target genes for this project.

In [ ]:
random.seed(13) 
target_genes = random.sample(all_genes, k=100)
print(len(target_genes))
target_genes[0:5]

100


['SERPINB1', 'KBTBD2', 'GCDH', 'ZNF493', 'MANBA']

## Gather all gene, chromosome, TSS start & end position

Now we have all the genes we are interested in. Let's prepare their corresponding chromosome, TSS start and end position information for future use.

Match all files, excluding gene_Y, gene_M, and gene_chr because they are either not useful or we do not have the matching bed file provided from the uchicago resource.

In [ ]:
path = "/content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_"
all_files = glob.glob(path + "*.txt") # match all files ending with gene_ 
all_files.remove('/content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_chr.txt') # remove because this is only a header
all_files.remove('/content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_Y.txt')  # remove because no bed file avaliable for this gene
all_files.remove('/content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_M.txt')  # remove because no bed file avaliable for this gene



We create a dataframe and store it if we have not yet. This dataframe contains all the gene information we have, specefically the chromosome the gene is located, the gene name, and its start and end position.

In [ ]:
gene_chrom_header = ['chr',	'gene_id','gene_name','start','end','gene_type'] # set a header
appended_data = []
for filenames in all_files:
  df = pd.read_csv(filenames ,header=None, names=gene_chrom_header, usecols=['chr','gene_name','start','end'] ,delimiter='\t')
    # store DataFrame in list
  appended_data.append(df)

df_full_gene_chrome = pd.concat(appended_data, ignore_index=True)

if not os.path.exists('/content/drive/MyDrive/Capstone_Project_2022/data/gene_chrom_all.csv'):
  df_full_gene_chrome.to_csv('/content/drive/MyDrive/Capstone_Project_2022/data/gene_chrom_all.csv')
else:
  df_full_gene_chrome = pd.read_csv('/content/drive/MyDrive/Capstone_Project_2022/data/gene_chrom_all.csv', index_col = 'Unnamed: 0')


This dataframe containg all genes' related info.

In [ ]:
df_full_gene_chrome.shape

(20025, 4)

In [ ]:
df_full_gene_chrome.head(3)

,chr,gene_name,start,end
0,10,TUBB8,92828,96053
1,10,ZMYND11,180405,300577
2,10,AL589988.1,225953,295049


Iterate through the target_genes list to get the info that we need and store it in drive.

In [ ]:
full_target_genes = []
for item in target_genes:
  df = df_full_gene_chrome.loc[df_full_gene_chrome.gene_name==item]
  full_target_genes.append(df)

df_full_target_genes = pd.concat(full_target_genes, ignore_index=True)

if not os.path.exists('/content/drive/MyDrive/Capstone_Project_2022/data/target_gene_info.csv'):
  df_full_target_genes.to_csv('/content/drive/MyDrive/Capstone_Project_2022/data/target_gene_info.csv')
else:
  df_full_target_genes = pd.read_csv('/content/drive/MyDrive/Capstone_Project_2022/data/target_gene_info.csv', index_col = 'Unnamed: 0')


In [ ]:
df_full_target_genes.shape

(100, 4)

In [ ]:
df_full_target_genes.head(3)

,chr,gene_name,start,end
0,6,SERPINB1,2832566,2842240
1,7,KBTBD2,32907784,32933743
2,19,GCDH,13001974,13010783


## Download bed file for each selected gene

The next lines of code will download the variation bed files, and we have created links to help us download the variation bed files for each chromosome, for each gene.

In [ ]:
chrom_bed_downloads = pd.read_csv("https://uchicago.box.com/shared/static/du77wf31li38tciv8imivwu57svae03p.csv")
chrom_bed_downloads.index = chrom_bed_downloads["chroms"]

chrom_bed_downloads.head(3) 

,chroms,link
chroms,,
1,1,https://uchicago.box.com/shared/static/9q9n4a0...
2,2,https://uchicago.box.com/shared/static/1tk6a3f...
3,3,https://uchicago.box.com/shared/static/77ldwqq...


We will define a function to help us download bed variation files for a given gene or list of genes

In [ ]:
def download_chrom_beds(chromosome, genes, downloads_table=chrom_bed_downloads):
  '''
  Downloads and unzip bed/variation files for a chromosome and list of genes
  '''

  link = downloads_table.loc[str(chromosome), "link"]
  print(link)
  chr_which = 'chr' + chromosome
  for gene in genes:
    if os.path.exists('/content/drive/MyDrive/Capstone_Project_2022/data/individual_beds/chr' + chromosome + '/chr' + chromosome + '_' + gene + '.bed'): # if the file is in the folder, no need to download again
      continue
    !curl -L {link} --output /content/drive/MyDrive/Capstone_Project_2022/data/chr_{chromosome}_bed.tar.gz && cd /content/drive/MyDrive/Capstone_Project_2022/data/ && tar -zxf /content/drive/MyDrive/Capstone_Project_2022/data/chr_{chromosome}_bed.tar.gz ./individual_beds/{chr_which}/{chr_which}_{gene}.bed

    # remove the download tar.gz file
    !rm /content/drive/MyDrive/Capstone_Project_2022/data/chr_{chromosome}_bed.tar.gz

The next step is to download the bed file for each selected gene. To do that we iterate the dataframe and use the previously defined function *download_chrom_beds*

In [ ]:
chromosomes = []
for index, row in df_full_target_genes.iterrows():
    chr = str(row['chr'])
    if chr not in chromosomes:
        chromosomes.append(chr)
    gene = [str(row['gene_name'])]
    path = local_path + '/data/individual_beds/' + 'chr' + chr + '/' + 'chr'+chr+'_'+gene[0]+'.bed'
    if not os.path.exists(path): # only download the bed files when they are not present in the drive
      download_chrom_beds(chromosome = chr, genes = gene)  

we read in a text file containing all the individual and their relationship with their population fron Geuvadis dataset. 

In [ ]:
geuvadis_relation = '/content/drive/MyDrive/Capstone_Project_2022/relationships_w_pops.txt'
geu_population = pd.read_csv(geuvadis_relation, sep='\t')
geu_population.head()

,FID,IID,dad,mom,sex,pheno,population
0,2357,NA19625,0,0,2,0,ASW
1,2367,NA19702,NA19700,NA19701,1,0,ASW
2,2367,NA19700,0,0,1,0,ASW
3,2367,NA19701,0,0,2,0,ASW
4,2368,NA19705,NA19703,NA19704,1,0,ASW


Create two sub-dataframe containing only CEU and YRI. These are the race we are interested in.

In [ ]:
CEU = geu_population.loc[geu_population.population=='CEU']
YRI = geu_population.loc[geu_population.population=='YRI']

## Check individual missingness and drop missing individuals form dataset

Now we are finally ready to select 120 individuals, however, before that we must check whether we have the individual in our bed file. This step is crucial otherwise we might select an individual that its haplotype information is missing on some genes which would cause us trouble!

In [ ]:
CEU_list = CEU['IID'].values.tolist()
YRI_list = YRI['IID'].values.tolist()

In [ ]:
# You may take a look at the bed file, which might help you to understand our logic
#path = "/content/drive/MyDrive/Capstone_Project_2022/data/individual_beds/chr1/chr1_CHI3L2.bed"
#!head $path

So, we define a function to help us check individual missingness

In [ ]:
def check_individuals(path_to_bed_file, list_of_individuals):

  '''
  Checks if an individual is missing in bed variation files. 
  These individuals should be removed prior to training
  returns all the missing individual in a list
  '''

  myfile = open(path_to_bed_file, 'r')
  myline = myfile.readline()
  bed_names = myline.split('\t')[4:]
  myfile.close()

  if set(list_of_individuals).issubset(set(bed_names)) == False:
    missing = list(set(list_of_individuals).difference(bed_names))
    #print('This (or these) individual(s) is/are not present: {}'.format(missing))
  else:
    missing = []
    #print('All individuals are present in the bed file.')

  return(missing)

In [ ]:
path = "/content/drive/MyDrive/Capstone_Project_2022/data/individual_beds/chr"
chromBed = []
for index, row in df_full_target_genes.iterrows():
    name = str(row['chr'])+'/chr'+str(row['chr'])+'_'+row['gene_name']
    chromBed.append(name)

CEU_Missing = set()
for item in chromBed:
  missing = check_individuals( path+chromBed[1]+'.bed', list_of_individuals=CEU_list)
  missing = set(missing)
  CEU_Missing.update(missing)

len(CEU_Missing) # this indicates how many individual are missing in all bed files 

91

In [ ]:
YRI_Missing = set()
for item in chromBed:
  missing = check_individuals( path+chromBed[1]+'.bed', list_of_individuals=YRI_list)
  missing = set(missing)
  YRI_Missing.update(missing)
len(YRI_Missing)

94

We drop the missing individuals and keep only the ones that are present in all bed files.

In [ ]:
YRI_set = set(YRI_list)
YRI_remain = YRI_set.difference(YRI_Missing)
len(YRI_remain)

86

In [ ]:
CEU_set = set(CEU_list)
CEU_remain = CEU_set.difference(CEU_Missing)
len(CEU_remain)

89

# Randomly Select Individual:

The goal of this section is to randomly select our 60 individuals among the remaining ones from the CEU and YRI population

First, we sort the remaining individual to give it an order, which makes our randomly select process more reliable.

In [ ]:
CEU_remain=list(CEU_remain)
CEU_remain.sort()
df_CEU_remain = pd.DataFrame(CEU_remain)

YRI_remain=list(YRI_remain)
YRI_remain.sort()
df_YRI_remain = pd.DataFrame(YRI_remain)

Save the selected individual in case the randome state does not work properly. But also just to keep track of the progress.

In [ ]:
path1 = "/content/drive/MyDrive/Capstone_Project_2022/data/CEU_sample_group.csv"
if not os.path.exists("/content/drive/MyDrive/Capstone_Project_2022/data/CEU_sample_group.csv"):
  CEU_sample_group = df_CEU_remain.sample(n=60, random_state=1)
  CEU_sample_group.columns = ['individual']
  CEU_sample_group.to_csv(path1, index=False)
  CEU_sample_group = pd.read_csv(path1)
else:
  CEU_sample_group = pd.read_csv(path1)

In [ ]:
CEU_sample_group.head(5)

,individual
0,NA12340
1,NA11931
2,NA12761
3,NA12750
4,NA12400


In [ ]:
path2 = "/content/drive/MyDrive/Capstone_Project_2022/data/YRI_sample_group.csv"
if not os.path.exists("/content/drive/MyDrive/Capstone_Project_2022/data/YRI_sample_group.csv"):
  YRI_sample_group = df_YRI_remain.sample(n=60, random_state=1)
  YRI_sample_group.columns = ['individual']
  YRI_sample_group.to_csv(path2, index = False)
  YRI_sample_group = pd.read_csv(path2)
else:
  YRI_sample_group = pd.read_csv(path2)

In [ ]:
YRI_sample_group.head(5)

,individual
0,NA19256
1,NA19092
2,NA19147
3,NA18488
4,NA19095


**Now we have selected our target genes and individuals, and we have also prepared the relevant files for them to move on to our next scipt where we will preform _gene expression prediction_ using Enformer.**